For each frame in the video is a node. Between every two nodes, there is a cost associated with
adding and removing apples. This cost is equal to the size of the exclusive or of the points in the two frames.
after calculating the cost between every two nodes, we can use the travelling salesman heuristic to find a path
through the frames that minimizes the number of apples added and removed.

In [46]:
import cv2
import numpy as np
from pathlib import Path
# add current directory to path
import sys
sys.path.append("~/src/badapple/badapple-irl/")
from python_tsp2.heuristics import solve_tsp_simulated_annealing, solve_tsp_lin_kernighan
from python_tsp2.exact import solve_tsp_dynamic_programming

In [3]:
# setup
badapple_path = Path(".").resolve().parent / "badapple-small.mp4"
square_side = 30
# skip to frame 364 because I already did every frame before that
frame_start = 365

width: 960, height: 720, fps: 30, n_frames: 6572


True

In [19]:
# get all the points in each frame
cap = cv2.VideoCapture(str(badapple_path))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"width: {width}, height: {height}, fps: {fps}, n_frames: {n_frames}")
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_start)

frame_count = 0
frame_points = []
original_frame_id_map = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if frame_count % 100 == 0:
        print(f"frame {frame_count}/{(n_frames-frame_start)//2}", end="\r")
    frame_count += 1

    current_frame_pixels = set()
    for i in range(0, width - 1, square_side):
        for j in range(0, height - 1, square_side):
            section = frame[j: j + square_side, i: i + square_side]
            if (np.mean(section)) < 50:
                current_frame_pixels.add((i, j))
    frame_points.append(current_frame_pixels)
    original_frame_id_map[len(frame_points) - 1] = frame_start + (frame_count-1) * 2  # trust
    # skip a frame because I don't want to do every frame
    ret, frame = cap.read()
num_frames = len(frame_points)
print(f"\nnum_frames: {num_frames}")

width: 960, height: 720, fps: 30, n_frames: 6572
frame 3100/3103
num_frames: 3104


In [22]:
# calculate the cost between every two frames (distance matrix), and then run TSP algorithm to get optimized order of frames
costs = np.zeros((num_frames, num_frames))
for i in range(num_frames):
    for j in range(num_frames):
        costs[i][j] = len(frame_points[i] ^ frame_points[j])
        costs[j][i] = costs[i][j]

RecursionError: maximum recursion depth exceeded

In [49]:
import sys
import time
print(costs)
sys.setrecursionlimit(100000) 
start_time = time.time()
permutation, distance = solve_tsp_lin_kernighan(costs, x0=list(range(num_frames)), verbose=True)
print(permutation)
print(f"Time taken: {time.time() - start_time}")
print("Distance: ", distance)


[[  0.  13.  32. ... 759. 759. 759.]
 [ 13.   0.  23. ... 746. 746. 746.]
 [ 32.  23.   0. ... 739. 739. 739.]
 ...
 [759. 746. 739. ...   0.   0.   0.]
 [759. 746. 739. ...   0.   0.   0.]
 [759. 746. 739. ...   0.   0.   0.]]
Current value: 128076.0; Ejection chain: 1
Current value: 128072.0; Ejection chain: 1
Current value: 128036.0; Ejection chain: 1
Current value: 127918.0; Ejection chain: 1
Current value: 127794.0; Ejection chain: 1
Current value: 127724.0; Ejection chain: 1
Current value: 127722.0; Ejection chain: 1
Current value: 127714.0; Ejection chain: 1
Current value: 127700.0; Ejection chain: 1
Current value: 127700.0; Ejection chain: 1
Current value: 127698.0; Ejection chain: 1
Current value: 127688.0; Ejection chain: 1
Current value: 127674.0; Ejection chain: 1
Current value: 127672.0; Ejection chain: 1
Current value: 127670.0; Ejection chain: 1
Current value: 127670.0; Ejection chain: 1
Current value: 127610.0; Ejection chain: 1
Current value: 127592.0; Ejection chain: 

In [55]:
# calculate the number of apple moves using the normal order and optimized order of frames
base_apple_moves = 0
for i in range(num_frames - 1):
    base_apple_moves += len(frame_points[i] ^ frame_points[i + 1])
print(f"Base apple moves: {base_apple_moves}")
optimized_apple_moves = 0
for i in range(num_frames - 1):
    optimized_apple_moves += len(frame_points[permutation[i]] ^ frame_points[permutation[i + 1]])
print(f"Optimized apple moves: {optimized_apple_moves}, percentage of original: {optimized_apple_moves / base_apple_moves * 100}%")

Base apple moves: 127317
Optimized apple moves: 120202, percentage of original: 94.41158682658248%


In [53]:
# generate new diff frames using the optimized order of frames
circle_frames_dir = Path("circle_frames_opt")
circle_frames_dir.mkdir(exist_ok=True)
diff_frames_dir = Path("diff_frames_opt")
diff_frames_dir.mkdir(exist_ok=True)
for index in range(num_frames):
    frame1 = np.ones((height, width, 3), dtype=np.uint8) * 255
    for i in range(0, width - 1, square_side):
        for j in range(0, height - 1, square_side):
            if (i, j) in frame_points[permutation[index]]:
                text_color = (255, 255, 255)
                cv2.circle(frame1, (i+square_side//2, j+square_side//2), square_side//2, 0, -1)
            else:
                text_color = 0
            cv2.putText(frame1, f"{j//square_side+1}", (i+square_side//3-1, j + 22), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
            cv2.putText(frame1, f"{i//square_side+1}", (i+square_side//3-1, j + 12), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
    cv2.imwrite(str(circle_frames_dir / f"{index}.png"), frame1)
   

In [56]:
# print permutation to a file
print(permutation)
with open("permutation.txt", "w") as f:
    for i in permutation:
        f.write(f"{i}\n")

[0, 1, 443, 442, 441, 440, 439, 2501, 2502, 2503, 2504, 2505, 2506, 2507, 2508, 2509, 2510, 2511, 2512, 2513, 2514, 2515, 2516, 2517, 2518, 2519, 2520, 48, 2521, 2522, 2523, 2524, 2525, 2526, 2527, 2528, 2529, 2530, 2531, 2540, 2554, 2545, 2780, 225, 224, 2755, 2756, 2757, 2758, 2759, 2760, 2761, 2762, 2763, 2764, 2539, 2543, 2765, 2784, 2785, 2786, 2787, 2788, 2789, 2790, 2791, 2792, 2793, 2794, 2795, 2796, 2797, 2798, 2799, 2800, 2801, 2802, 2803, 2804, 2805, 2806, 2807, 2808, 2809, 2810, 2811, 2812, 2813, 2814, 2815, 2816, 2817, 2818, 2819, 2820, 2821, 2822, 2823, 2824, 2825, 2826, 2827, 2828, 2829, 2830, 2831, 2832, 2833, 2834, 2835, 2836, 2837, 2838, 2839, 2840, 2841, 2842, 2843, 2844, 2845, 2846, 2847, 2848, 2849, 2850, 2851, 2852, 2853, 2854, 2855, 2856, 2857, 2858, 2859, 2860, 2861, 2862, 2863, 2864, 2865, 2866, 552, 553, 554, 1679, 1678, 1677, 1676, 1675, 1674, 1673, 1672, 1671, 1670, 1669, 1668, 1667, 1666, 1665, 1664, 1663, 1662, 1661, 1660, 1659, 1658, 1657, 1656, 1655, 165

In [54]:
# generate diff frames by comparing current frame with previous frame's set, making green for additions and red for removals

last_frame_pixels = set()
for index in range(num_frames-1):
        
    out_frame = np.ones((height, width, 3), dtype=np.uint8) * 255
    current_frame_pixels = frame_points[permutation[index]]
    
    for i in range(0, width - 1, square_side):
        for j in range(0, height - 1, square_side):
            if (i, j) in current_frame_pixels and (i, j) in last_frame_pixels:
                # persisted from previous frame. Black circle, white text
                circle_colour = (0, 0, 0)
                text_color = (255, 255, 255)
            elif (i, j) in current_frame_pixels:
                # new black dot. Green circle, black text
                circle_colour = (0, 255, 0)
                text_color = (0, 0 ,0)
            elif (i, j) in last_frame_pixels:
                # removed black dot. Red circle, black text
                circle_colour = (0, 0, 255)
                text_color = (0, 0, 0)
            else:
                # persisted nothing. Just use black text
                text_color = (0, 0 ,0)
                circle_colour = None
            if circle_colour is not None:
                cv2.circle(out_frame, (i+square_side//2, j+square_side//2), square_side//2, circle_colour, -1)
            cv2.putText(out_frame, f"{j//square_side+1}", (i+square_side//3-1, j + 22), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
            cv2.putText(out_frame, f"{i//square_side+1}", (i+square_side//3-1, j + 12), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)

    cv2.imwrite(f"diff_frames_opt/{index}.png", out_frame)

    last_frame_pixels = current_frame_pixels